In [1]:
import csv
import datetime


def parse_sp500_date(date_str):
    """
    Parses a date string in M/D/YYYY format into a datetime.date object.
    """
    return datetime.datetime.strptime(date_str, "%m/%d/%Y").date()


def parse_gold_date(date_str):
    """
    Tries to parse the 'Date' in the gold file.
    If it's only a year (e.g. '1833'), we return None, indicating it's year-only data.
    Otherwise, return a datetime.date object.
    """
    # If the date string is purely digits (e.g., '1833'), treat it as a year
    if date_str.isdigit():
        return None
    # Otherwise, parse as M/D/YYYY
    return datetime.datetime.strptime(date_str, "%m/%d/%Y").date()


def read_sp500_data(filename):
    """
    Reads S&P 500 data from CSV (Date,Value) into a dict: { date_obj: value_float }.
    """
    sp500_dict = {}
    with open(filename, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            date_str = row["Date"]
            value_str = row["Value"].replace(
                ",", ""
            )  # remove commas from numbers like "5,917.02"

            try:
                date_obj = parse_sp500_date(date_str)
                value = float(value_str)
                sp500_dict[date_obj] = value
            except ValueError:
                # skip lines that can't be parsed
                continue
    return sp500_dict


def read_gold_data(filename):
    """
    Reads Gold data from CSV (Date,Value).
    Returns a dictionary: { date_obj: gold_price } for all monthly/daily data,
    plus fills in year-based data as needed.
    """
    # Step 1: read rows into two buckets: (a) exact-dates and (b) year-only
    exact_dates = {}  # { date_obj: price }
    yearly_averages = {}  # { year_int: price }

    with open(filename, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            raw_date = row["Date"]
            value_str = row["Value"].replace(",", "")

            try:
                gold_value = float(value_str)
            except ValueError:
                continue  # skip lines that can't be parsed

            # Try parsing the date
            parsed = parse_gold_date(raw_date)

            if parsed is None:
                # Means raw_date is just a year
                # store year-based average
                year_int = int(raw_date)
                yearly_averages[year_int] = gold_value
            else:
                # We have a month/day/year date
                exact_dates[parsed] = gold_value

    # Step 2: For each year-based entry, fill in monthly placeholders if exact dates are missing
    # We will create a dictionary gold_data with a daily date_obj for each month that is missing.
    gold_data = {}

    # First, copy over the exact monthly/daily data we have
    for d_obj, val in exact_dates.items():
        gold_data[d_obj] = val

    # Then fill in year-based placeholders
    for year_int, val in yearly_averages.items():
        # We will iterate over all 12 months and set the 1st of each month
        # if that date is not already in gold_data
        for month in range(1, 13):
            # We'll just pick day=1 for placeholders
            placeholder_date = datetime.date(year_int, month, 1)

            if placeholder_date not in gold_data:
                gold_data[placeholder_date] = val

    return gold_data


def unify_data(sp500_dict, gold_dict, output_filename):
    """
    Merges S&P and Gold data on the earliest date in sp500_dict through the latest date in sp500_dict.
    For each date in S&P data, tries to find a gold price. If gold price not found for that exact day,
    tries to find the 1st of the month or else defaults to year average.
    (But we already prefilled gold_dict with year placeholders, so we just need the same date.)
    """
    # We'll collect (date, sp_val, gold_val) and then write them out
    merged_rows = []

    for sp_date in sorted(sp500_dict.keys()):
        sp_val = sp500_dict[sp_date]

        # We try an exact match for sp_date in gold_dict
        gold_val = gold_dict.get(sp_date, None)
        if gold_val is not None:
            merged_rows.append((sp_date, sp_val, gold_val))
        else:
            # If we didn't find a match (maybe S&P date is mid-month),
            # try date’s year-month with day=1.
            # Because we filled placeholders for the 1st of each month, we can do that:
            fallback_date = datetime.date(sp_date.year, sp_date.month, 1)
            gold_val = gold_dict.get(fallback_date, None)

            # If still None, try year average fallback_date= Jan 1st
            if gold_val is None:
                fallback_date = datetime.date(sp_date.year, 1, 1)
                gold_val = gold_dict.get(fallback_date, None)

            # If still None, we just skip or use some placeholder (e.g. None)
            if gold_val is None:
                # you can decide to skip or set to None
                gold_val = None

            merged_rows.append((sp_date, sp_val, gold_val))

    # Write out to CSV
    with open(output_filename, "w", newline="", encoding="utf-8") as out:
        writer = csv.writer(out)
        writer.writerow(["Date", "SP500", "Gold"])

        for date_obj, sp_val, gold_val in merged_rows:
            date_str = date_obj.strftime("%m/%d/%Y")
            writer.writerow([date_str, sp_val, gold_val])


def main():
    # Paths to your input files
    sp500_file = "raw_sp500_price_history.csv"
    gold_file = "raw_gold_price_history.csv"
    output_file = "cleaned_finance_data.csv"

    # Read the data
    sp500_dict = read_sp500_data(sp500_file)
    gold_dict = read_gold_data(gold_file)

    # Merge and write to a single CSV
    unify_data(sp500_dict, gold_dict, output_file)
    print(f"Data unified and written to {output_file}")


if __name__ == "__main__":
    main()

Data unified and written to cleaned_finance_data.csv


# Data Cleaning and Unification Script Documentation

## Overview

This script is designed to unify two datasets: S\&P 500 historical price data and gold historical price data. It processes the raw data, fills in gaps for gold prices (where only yearly averages are available), and creates a single unified CSV file with corresponding S\&P 500 and gold prices. This output is ready for analysis and visualization.

The datasets were sourced from the following:
1. **S\&P 500 Data**: Retrieved from [multpl.com](https://www.multpl.com/s-p-500-historical-prices), which provides historical monthly prices for the index.
2. **Gold Data**: Compiled from two sources:
    * [Historic Gold Prices 1833-Present (National Mining Association)](https://nma.org/wp-content/uploads/2016/09/historic_gold_prices_1833_pres.pdf)
    * Modern gold price data from Yahoo Finance (symbol: GC=F).

This documentation explains how the code processes, merges, and handles missing data in these datasets.

## Script Functionality

1. **File Input**
    * **S\&P 500 File** (sp500.csv): Contains columns Date (in MM/DD/YYYY format) and Value (formatted as strings with commas, e.g., "5,917.02").
    * **Gold File** (gold.csv): Contains columns Date (which may be either a year or a specific MM/DD/YYYY date) and Value (price in numeric format).
2. **Data Processing**
    * **S\&P 500 Data**:
        * Parses dates into datetime.date objects.
        * Converts price values to float, removing commas.
        * Creates a dictionary {date: price} for efficient lookups.
    * **Gold Data**:
        * Parses the Date column:
        * If the value is a year (e.g., 1833), it treats the price as the annual average and assigns it to all months within that year (January–December).
        * If the value is a specific date (MM/DD/YYYY), it associates the exact date with the price.
        * Handles yearly averages by creating placeholder entries for the 1st day of each month within the year if specific monthly/daily data is missing.
        * Creates a dictionary {date: price} where date is a datetime.date object.
3. **Merging Data**
    * For each date in the S\&P 500 data, the script:
    * Attempts to find a matching gold price for the exact date.
    * If no exact match exists, it defaults to the 1st of the month, or falls back to the year average.
    * Produces a unified dataset with the structure: `Date,SP500,Gold`
4. **Output**
    * Writes the unified data to a CSV file (unified\_data.csv).
    * Dates are formatted as MM/DD/YYYY, and prices are represented as floats.

## How to Use

1. Prepare your datasets in the following formats:
    * **S\&P 500 Data**: A CSV file named sp500.csv with columns Date and Value.
    * **Gold Data**: A CSV file named gold.csv with columns Date and Value.
    * The Date column may include years (e.g., 1833) or full dates (MM/DD/YYYY).
    * Ensure all prices are numerical or strings that can be converted to floats.
2. Place the script and the input files (sp500.csv and gold.csv) in the same directory.
3. Run the script. The output will be saved as unified\_data.csv.
4. Examine the unified CSV for analysis.

## Assumptions and Notes

* **Fallback Logic**:
    * Gold prices default to the 1st of the month if no specific date is available.
    * If no monthly data is available, it defaults to the annual average.
* **Data Cleaning**:
    * The script removes commas from numerical values and skips invalid rows (e.g., missing or unparsable dates/values).
* **Granularity**:
    * The unified dataset retains daily granularity where available but gracefully handles missing data.

## Recreating the Data

If you’d like to recreate the data yourself:

* **S\&P 500 Historical Data**: Download from [multpl.com](https://www.multpl.com/s-p-500-historical-prices).
* **Gold Historical Data**:
    * Use the PDF provided by the [National Mining Association](https://nma.org/wp-content/uploads/2016/09/historic_gold_prices_1833_pres.pdf) for historical averages.
    * Retrieve recent gold prices from Yahoo Finance (GC=F symbol) or another financial data source.

You can follow a similar approach to process and merge the data into a unified format.